In [8]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain.agents import load_tools, initialize_agent, AgentExecutor, create_openai_functions_agent


In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
MODEL = "gpt-3.5-turbo"
llm = ChatOpenAI(model=MODEL, temperature=0)

In [30]:
query = """
    Please provide the following details:
    1. What is the capital of Australia?
    2. Who is the current President of the United States?
    3. What is the population of the city where the President of the United States was born?
    4. What is the GDP of the country of which the capital is the birthplace of the current President of the United States?
    5. What are the main industries in the birthplace city of the current President of the United States?
    6. Who was the President of the United States before the current one and where was he born?
"""


In [31]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults()]

In [32]:
tools = load_tools(["google-serper", "llm-math"], llm=llm)
react_agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [33]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a general agent."),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
agent = AgentExecutor(
    agent=create_openai_functions_agent(llm, tools, prompt), 
    tools=tools, 
    handle_parsing_errors=True, 
    verbose=False,
)

In [34]:
response = react_agent.invoke(query)
print(response['output'])



> Entering new AgentExecutor chain...
I need to gather information on the capital of Australia, the current President of the United States, and related details.
Action: google_serper
Action Input: "capital of Australia"
Observation: Australian Capital Territory: Australian Territory. The Australian Capital Territory houses Canberra, Australia’s capital, built between Sydney and Melbourne in the early 20th century. The federal district’s forest, farmland and nature reserves earn Canberra its nickname, the "Bush Capital.” The... Australian Capital Territory Capital: Canberra. Australian Capital Territory Population: 431,215 (Sep 2020). Australian Capital Territory Area: 910 mi². Australian Capital Territory Chief minister: Andrew Barr. Australian Capital Territory Bird: Gang-gang cockatoo. Australian Capital Territory Colour(s): Blue and gold. Australian Capital Territory HDI (2021): 0.980; very high · 1st. The capital city was founded and formally named as Canberra in 1913. A plan by 

In [35]:
response = agent.invoke({
    "messages": [
        HumanMessage(content=query),
    ],
})

print(response['output'])

6. The President of the United States before Joe Biden was Donald Trump. He was born in Queens, New York City, New York.

If you have any more questions or need further information, feel free to ask!
